<h1>Testing the E2E simulations - initial try</h1>

This was done before I had developped the `SegmentedTelescopeAPLC` class.

## -- ATLAST aperture --

This script introduces the end-to-end (E2E) simulations that are used in **`calibration.py`**, for the influence calibration of each individual segment. The testing of the script itself is done in this next notebook.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
%matplotlib inline
from astropy.io import fits
import astropy.units as u
from hcipy import *

os.chdir('../../pastis/')
from config import CONFIG_PASTIS
import util as util
import image_pastis as impastis
import atlast_imaging as atim

# Path to all the outputs from "aperture_definition.py".
dir = '/Users/ilaginja/Documents/data_from_repos/pastis_data/active/calibration'
# Change into that directory
os.chdir(dir)

In [ ]:
# Get some parameters
which_tel = CONFIG_PASTIS.get('telescope', 'name')
im_lamD = 30     # image size in lambda/D
nb_seg = CONFIG_PASTIS.getint(which_tel, 'nb_subapertures')
zern_max = CONFIG_PASTIS.getint('zernikes', 'max_zern')
inner_wa = CONFIG_PASTIS.getint(which_tel, 'IWA')
outer_wa = CONFIG_PASTIS.getint(which_tel, 'OWA')
sampling = CONFIG_PASTIS.getfloat(which_tel, 'sampling')            # sampling
tel_size_px = CONFIG_PASTIS.getint('numerical', 'tel_size_px')
tel_size_px = 614  #### THIS NEEDS TO BE THE SAME SIZE LIKE APODIZER ARRAY. I asked Remi for a 1024 x 1024 one.
diam = CONFIG_PASTIS.getfloat(which_tel, 'diameter')
wvln = CONFIG_PASTIS.getint(which_tel, 'lambda') * u.nm

nm_aber = CONFIG_PASTIS.getfloat('calibration', 'single_aberration') * u.nm   # [nm] amplitude of aberration
zern_number = CONFIG_PASTIS.getint('calibration', 'zernike')                  # Which (Noll) Zernike we are calibrating for

In [ ]:
# Print some of the loaded parameters
print('Telescope: {}'.format(which_tel))
print('Image size: {} lambda/D'.format(im_lamD))
print('Pupil in pixels: {}'.format(tel_size_px))
print('Number of segments: {}'.format(nb_seg))
print('IWA: {} lambda/D'.format(inner_wa))
print('OWA: {} lambda/D'.format(outer_wa))
print('Sampling: {}'.format(sampling))
print('Diameter: {} m'.format(diam))
print('Wavelength: {}'.format(wvln))

print('\nAberration: {}'.format(nm_aber))
print('Zernike: {}'.format(zern_number))

For starters, lets completely independently create some HCIPy images of a direct image (no coronagraph) and a coronagraphic image.

In [ ]:
# HCIPy grids and propagator
pupil_grid = make_pupil_grid(dims=tel_size_px, diameter=diam)
focal_grid = make_focal_grid_from_pupil_grid(pupil_grid, sampling, im_lamD, wavelength=wvln.to(u.m).value)
prop = FraunhoferPropagator(pupil_grid, focal_grid)

In [ ]:
# Generate an aperture
aper, seg_pos = atim.get_atlast_aperture(normalized=False)
aper = evaluate_supersampled(aper, pupil_grid, 2)

plt.figure(figsize=(15, 15))
imshow_field(aper)
plt.title('ATLAST aperture')

The specs for image size will require a general overhaul, but at this point I'm just doing whatever makes this work.

Lets also instanciate a segmented mirror that we can control in piston, tip and tilt.

In [ ]:
# Instantiate the segmented mirror
sm = atim.SegmentedMirror(aper, seg_pos)

# Make a pupil plane wavefront from aperture
wf_pre_sm = Wavefront(aper, wavelength=wvln.to(u.m).value)

# Apply SM if you want to
wf_post_sm = sm(wf_pre_sm)

imshow_field(wf_post_sm.intensity)
plt.title('Wavefront intensity at HCIPy SM')

Let's make sure the segment controls work.

In [ ]:
sm.flatten()
for i in [19, 28, 4, 36]:
    sm.set_segment(i, 1e-8, 0, 0)   # PTT in meters and radians
    
# Make throw-away wavefront that we will display here just to make sure the SM works.
wf_throw = sm(wf_pre_sm)

imshow_field(wf_throw.phase, mask=aper, cmap='RdBu')
plt.title('Checking the SM')
plt.colorbar()

In [ ]:
# Reset the SM
sm.flatten()

## NO CORONAGRAPH

### Generating a direct PSF without aberrations

In [ ]:
# Propagate from (flat) SM to image plane
im_ref = prop(wf_post_sm)

# Display
imshow_field(np.log10(im_ref.intensity/np.max(im_ref.intensity)))
plt.title('Intensity of flat SM')
plt.colorbar()

We want to have normalized images, normalized to the non-coronagraphic, non-aberrated (meaning no segment is actively moved) PSF that is displayed above. **normp** will be our normalization factor.

In [ ]:
normp = np.max(im_ref.intensity)

### A single aberrated segment

To compare to the analytical images step by step, I will first create images with only one segment aberrated.

In [ ]:
segnum = 5     # Which segment are we aberrating - numbered like in the HCIPy aperture

nm_aber = 100 * u.nm

print('Aberrated segment:', segnum)

Our input aberration is in nanometers of OPD, while HCIPy needs it in meters of surface error. This means that I have to divide the input aberration by 2 before applying it to the segmented mirror `sm`.

In [ ]:
# Crate OPD with aberrated segment, NO CORONAGRAPH
print('Applying aberration to OTE.')
print('nm_aber: {}'.format(nm_aber))
sm.flatten()   # Making sure there are no previous movements on the segments.
sm.set_segment(segnum, nm_aber.to(u.m).value/2, 0, 0)

In [ ]:
# Display the HCIPy pistoned segments
wf_pistoned = sm(wf_pre_sm)

imshow_field(wf_pistoned.phase, mask=aper, cmap='RdBu')
plt.title('Phase for single aberrated segment')
plt.colorbar()

In [ ]:
# Propagate from SM to image plane
im_pistoned = prop(wf_pistoned)

# Display
imshow_field(np.log10(im_pistoned.intensity/normp))
plt.title('Image for single pistoned segment')
plt.colorbar()

Compare this image with one single aberrated segment vs. the non-aberrated PSF:

In [ ]:
# Subtract the perfect direct PSF off the single-segment aberrated PSF
one_aber_residual = im_ref.intensity - im_pistoned.intensity

plt.figure(figsize=(20, 10))
plt.subplot(1, 3, 1)
imshow_field(np.log10(im_ref.intensity/normp))
plt.title('Direct PSF, perfect')
plt.colorbar()

plt.subplot(1, 3, 2)
imshow_field(np.log10(im_pistoned.intensity/normp))
plt.title('Direct PSF one aberrated segment')
plt.colorbar()

plt.subplot(1, 3, 3)
imshow_field(np.log10(one_aber_residual))
plt.title('Residual')
plt.colorbar()
plt.show()

While not much seems to change for an aberration of 1 nm, we can see in the residual image that there is indeed a difference between the aberrated and non-aberrated PSF. When we give it an aberration of 0 nm, residual image is completely blank. When we use an aberration of 100 nm, the difference becomes very obvious.

### Pair-wise aberrated segments

In [ ]:
# Decide which two segments you want to aberrate
segnum1 = 22     # Which segments are we aberrating - numbering after HCIPy's aperture numbering
segnum2 = 15
seg_array = np.array([segnum1, segnum2])

zern_pair = 1  # Which Noll Zernike are we putting on the segments.
nm_aber = 200 * u.nm

print('Aberration: {}'.format(nm_aber))
print('Aberrated segments:', seg_array)
print('Noll Zernike used:', zern_pair)

In [ ]:
# Apply aberration to all segments
sm.flatten()   # Making sure there are no previous movements on the segments.
for nseg in seg_array:
    # Put Zernike on correct segments on OTE
    sm.set_segment(nseg, nm_aber.to(u.m).value/2, 0, 0)

# Display the OTE
wf_pairs = sm(wf_pre_sm)

imshow_field(wf_pairs.phase, mask=aper, cmap='RdBu')
plt.title('Phase for pair of aberrated segments')
plt.colorbar()

In [ ]:
# Display the propagated image
im_pairs = prop(wf_pairs)

imshow_field(np.log10(im_pairs.intensity/normp))
plt.title('Direct PSF of a pair-wise aberrated segmented OTE')
plt.colorbar()

print(im_pairs.intensity.shaped.shape)

In [ ]:
im = util.zoom_cen(im_pairs.intensity.shaped, 40)
plt.imshow(np.log10(im))

Hmmm these images do show fringes, but they look different from the ones in the analytical images. Instead of a prominent display of dark fringes, it's very brings fringes that start showing up. The Poppy HexDM images show the same behavior, so it's not about the simulator, but I wonder why this is the case.

I created some images from specific pairs in the analytical model too and then saved them:

In [ ]:
# PISTON - Noll 1
#segs_1_2_noll_1_dir = np.copy(im_pairs.intensity.shaped)
#segs_1_10_noll_1_dir = np.copy(im_pairs.intensity.shaped)
#segs_1_24_noll_1_dir = np.copy(im_pairs.intensity.shaped)
#segs_1_7_noll_1_dir = np.copy(im_pairs.intensity.shaped)
#segs_1_19_noll_1_dir = np.copy(im_pairs.intensity.shaped)
#segs_19_28_noll_1_dir = np.copy(im_pairs.intensity.shaped)

In [ ]:
save_dir1 = '/astro/opticslab1/PASTIS/atlast_data/E2E_pair_aberrations/2019-5-6-16h-00min_piston_300nm'
# util.write_fits(segs_1_2_noll_1_dir, os.path.join(save_dir1, 'segs_1_2_noll_1_dir.fits'))
# util.write_fits(segs_1_10_noll_1_dir, os.path.join(save_dir1, 'segs_1_10_noll_1_dir.fits'))
# util.write_fits(segs_1_24_noll_1_dir, os.path.join(save_dir1, 'segs_1_24_noll_1_dir.fits'))
# util.write_fits(segs_1_7_noll_1_dir, os.path.join(save_dir1, 'segs_1_7_noll_1_dir.fits'))
# util.write_fits(segs_1_19_noll_1_dir, os.path.join(save_dir1, 'segs_1_19_noll_1_dir.fits'))
# util.write_fits(segs_19_28_noll_1_dir, os.path.join(save_dir1, 'segs_19_28_noll_1_dir.fits'))

In general, I will have to load these images from central store:
- '/astro/opticslab1/PASTIS/jwst_data/E2E_pair_aberrations/2019-1-18-17h-5min_piston_1000nm_pairs' will have images generated with aberrations of 1000 nm per segment which is too much compared to JWST's wavelength and the sort of aberrations that are expected in-flight
- '/astro/opticslab1/PASTIS/jwst_data/E2E_pair_aberrations/2019-1-25-16h-18min_piston_100nm' has images generated with aberrations of 100 nm per segment, but this aberration is not high enough to make us see the fringes


In [ ]:
read_dir1 = '/astro/opticslab1/PASTIS/atlast_data/E2E_pair_aberrations/2019-5-6-16h-00min_piston_300nm'
segs_1_2_noll_1_dir = fits.getdata(os.path.join(read_dir1, 'segs_1_2_noll_1_dir.fits'))
segs_1_10_noll_1_dir = fits.getdata(os.path.join(read_dir1, 'segs_1_10_noll_1_dir.fits'))
segs_1_24_noll_1_dir = fits.getdata(os.path.join(read_dir1, 'segs_1_24_noll_1_dir.fits'))
segs_1_7_noll_1_dir = fits.getdata(os.path.join(read_dir1, 'segs_1_7_noll_1_dir.fits'))
segs_1_19_noll_1_dir = fits.getdata(os.path.join(read_dir1, 'segs_1_19_noll_1_dir.fits'))
segs_19_28_noll_1_dir = fits.getdata(os.path.join(read_dir1, 'segs_19_28_noll_1_dir.fits'))

Let's have a look at some of the images (refer to numbered pupil to identify baslines these correspond to).

In [ ]:
print('Loaded images shape:', segs_1_2_noll_1_dir.shape)

plt.figure(figsize=(18, 12))
plt.suptitle('Pair-wise aberrations on direct (no coro) HCIPy images')
plt.subplot(2, 3, 1)
plt.imshow(np.log10(segs_1_2_noll_1_dir), norm=LogNorm())
plt.title('Piston on segments 1 and 2')

plt.subplot(2, 3, 2)
plt.imshow(np.log10(segs_1_10_noll_1_dir), norm=LogNorm())
plt.title('Piston on segments 1 and 10')

plt.subplot(2, 3, 3)
plt.imshow(np.log10(segs_1_24_noll_1_dir), norm=LogNorm())
plt.title('Piston on segments 1 and 24')

plt.subplot(2, 3, 4)
plt.imshow(np.log10(segs_1_7_noll_1_dir), norm=LogNorm())
plt.title('Piston on segments 1 and 7')

plt.subplot(2, 3, 5)
plt.imshow(np.log10(segs_1_19_noll_1_dir), norm=LogNorm())
plt.title('Piston on segments 1 and 19')

plt.subplot(2, 3, 6)
plt.imshow(np.log10(segs_19_28_noll_1_dir), norm=LogNorm())
plt.title('Piston on segments 19 and 28')

plt.show()

I made sure to make images from the same aberrated pairs like in the analytical notebook (notebook 2), so we can compare them here now.

In [ ]:
# Load the analytical images
read_dir_ana = '/astro/opticslab1/PASTIS/atlast_data/uncalibrated_analytical_images/2019-05-06-11h-30min_1nm_piston'
segs_1_2_noll_1_ana = fits.getdata(os.path.join(read_dir_ana, 'segs_1_2_noll_1.fits'))
segs_1_10_noll_1_ana = fits.getdata(os.path.join(read_dir_ana, 'segs_1_10_noll_1.fits'))
segs_1_24_noll_1_ana = fits.getdata(os.path.join(read_dir_ana, 'segs_1_24_noll_1.fits'))
segs_1_7_noll_1_ana = fits.getdata(os.path.join(read_dir_ana, 'segs_1_7_noll_1.fits'))
segs_1_19_noll_1_ana = fits.getdata(os.path.join(read_dir_ana, 'segs_1_19_noll_1.fits'))
segs_19_28_noll_1_ana = fits.getdata(os.path.join(read_dir_ana, 'segs_19_28_noll_1.fits'))

Compare pairs **3-11**, **6-11** and **11-17** between E2E and analytical:

In [ ]:
plt.figure(figsize=(18, 12))
plt.suptitle('Comparison of E2E and analtical DIRECT images')

plt.subplot(2, 3, 1)
plt.imshow(np.log10(segs_1_2_noll_1_dir), norm=LogNorm())
plt.title('Piston on segments 1 and 2 - E2E')

plt.subplot(2, 3, 2)
plt.imshow(np.log10(segs_1_10_noll_1_dir), norm=LogNorm())
plt.title('Piston on segments 1 and 10 - E2E')

plt.subplot(2, 3, 3)
plt.imshow(np.log10(segs_1_24_noll_1_dir), norm=LogNorm())
plt.title('Piston on segments 1 and 24 - E2E')

plt.subplot(2, 3, 4)
plt.imshow(np.log10(segs_1_2_noll_1_ana), norm=LogNorm())
plt.title('Piston on segments 1 and 2 - analytical')

plt.subplot(2, 3, 5)
plt.imshow(np.log10(segs_1_10_noll_1_ana), norm=LogNorm())
plt.title('Piston on segments 1 and 10 - analytical')

plt.subplot(2, 3, 6)
plt.imshow(np.log10(segs_1_24_noll_1_ana), norm=LogNorm())
plt.title('Piston on segments 1 and 24 - analytical')

Something is not quite right. Here are two more pairs.

In [ ]:
plt.figure(figsize=(15, 15))
plt.suptitle('Comparison of E2E and analtical DIRECT images')

plt.subplot(2, 2, 1)
plt.imshow(np.log10(segs_1_7_noll_1_dir), norm=LogNorm())
plt.title('Piston on segments 1 and 7 - E2E')

plt.subplot(2, 2, 2)
plt.imshow(np.log10(segs_1_19_noll_1_dir), norm=LogNorm())
plt.title('Piston on segments 1 and 19 - E2E')

plt.subplot(2, 2, 3)
plt.imshow(np.log10(segs_1_7_noll_1_ana), norm=LogNorm())
plt.title('Piston on segments 1 and 7 - analytical')

plt.subplot(2, 2, 4)
plt.imshow(np.log10(segs_1_19_noll_1_ana), norm=LogNorm())
plt.title('Piston on segments 1 and 19 - analytical')

## WITH CORONAGRAPH

### Generating a coronagraphic PSF without aberrations

#### Perfect coronagraph

To use a **perfect coronagraph** in HCIPy, we generate a new propagator instead of the normal Fraunhofer propagator, and it will be defined on an aperture.

In [ ]:
# Create coronagraph propagator on aperture
coro_prop = PerfectCoronagraph(aper)

We can then propagate the E-field from the pupil plane on the SM to the Lyot plane.

In [ ]:
# Propagate from the wavefront at the SM to Lyot plane
lyot_plane = coro_prop(wf_post_sm)

# Display the Lyot plane
imshow_field(np.log10(lyot_plane.intensity))
plt.colorbar()

We can see that there is essentially no light anywhere, since we're using a perfect coronagraph.

Lets still apply a Lyot stop just to see how to do that and then propagate with the Fraunhofer propagator to the final image plane. Now, currently I can only add an annular Lyot stop, but I will have to figure out a way of applying a real Lyot stop.

The `Apodizer()` acts as a mask, and is basically there to have something (a functioning Python object within the HCIPy framework) to propagate a Wavefront through. It can simulate any transmissive or reflective mask, such as a pupil mask, a Lyot stop, a printed and optimized apodizer as the ones you have in the APLC, etc.

In [ ]:
# Create the Lypt stop
lyot_x = circular_aperture(0.99*diam)(pupil_grid)
lyot_stop = Apodizer(lyot_x)

# Display the Lyot stop
imshow_field(lyot_x)

In [ ]:
#Show the overlap of the Lyot stop and the aperture
imshow_field(lyot_x + aper)
plt.title('Aperture and Lyot stop overlap')

In [ ]:
# Apply the Lyot stop to the Lyot plane and propagate to the image plane
im_coro_perf = prop(lyot_stop(lyot_plane))

# Display the coro image
imshow_field(np.log10(im_coro_perf.intensity / normp), vmin=1e-12)
plt.title("Perfect coronagraph, no aberrations")
plt.colorbar()

And there is virtually no light in the image since a perfect coronagraphs cancels it out completely by definition.

Note how I needed to set a scale min/max of $10^{-12}$ to get an empty image. If I let it autoscale, it weill rescale on a light leel of lower than $10^{-30}$ - and show a full PSF. This is fine, because this still means that there is basically no light in the image.

#### Lyot coronagraph

Lets try to look at something more realistic and use a Lyot coronagraph.

We will again have to define a new propagator, this time for the Lyot coronagraph, but first we need to generate a focal plane mask (FPM). The radius of the FPM is given in $radians$ (because everything in HCIPy is in SI units). If we had a system of $\lambda=1$ and $D=1$, then $1 \lambda/D = 1 rad$. Since this is not the case here though, and we only know our FPM radius in terms of $\lambda/D$, we need to figure out what angle that is in the focal plane.

If $\lambda = 638 \times 10^{-9}$ and $D=15m$, then $\lambda/D = \frac{638 \times 10^{-9}}{15} = 4.25 \times 10^{-8} rad$.

In [ ]:
lam_over_d = wvln.to(u.m).value / diam  # rad
print('lambda/D = {}'.format(lam_over_d))

# Create FPM on a focal grid, with radius in lambda/D, converted to radians
fpm = 1 - circular_aperture(8*lam_over_d)(focal_grid)   # circular_aperture takes diameter
print(fpm.shaped.shape)

# Display the FPM
imshow_field(fpm)
plt.title('Focal plane mask')

A neat way of making sure the FPM is really around the size we want it to be is to put it in the focal plane reference image of the aperture we're working with.

In [ ]:
imshow_field(fpm * im_ref.intensity/normp, norm=LogNorm())

Now we can generate a Lyot coronagraph propagator with the pupil grod of the aperture and the FPM as inputs.

In [ ]:
# Create Lyot coronagraph on pupil grid with FPM
coro_lyot = LyotCoronagraph(pupil_grid, fpm)

And now we can propagate the wavefront at the SM through the coronagraph to the image plane.

In [ ]:
# Propagate from the wavefront at the SM to Lyot plane
lyot_plane = coro_lyot(wf_post_sm)

# Display the Lyot plane
imshow_field(np.log10(lyot_plane.intensity))
plt.colorbar()

Now this looks a lot more realistic and it also becomes evident why we need a proper Lyot stop. While I might need some fancier solutions in the future, I can make a Lyot stop with a central obscuration for now, almost HiCAT-like.

In [ ]:
aperture = circular_aperture(0.9 * diam)
obscuration = circular_aperture(0.2 * diam)
res = aperture(pupil_grid) - obscuration(pupil_grid)
ls = Field(res, pupil_grid)

imshow_field(ls)
plt.title('Lyot stop')

In [ ]:
# Display the Lyot stop in the Lyot plane
imshow_field(res * lyot_plane.intensity/normp, norm=LogNorm())

In [ ]:
# Propagate to final image
lyot_stop = Apodizer(aperture(pupil_grid) - obscuration(pupil_grid))

# Apply the Lyot stop to the Lyot plane and propagate to the image plane
im_coro_lyot = prop(lyot_stop(lyot_plane))

# Display the coro image
imshow_field(np.log10(im_coro_lyot.intensity / normp))
plt.title("Lyot coronagraph, no aberrations")
plt.colorbar()

Let's compare this to the reference image.

In [ ]:
imshow_field(im_ref.intensity/normp, norm=LogNorm())
plt.title('No coro, no aberration')
plt.colorbar()

Is the Lyot coronagraph any better than the reference image? I couldn't tell. Let's try plugging in the APLC apodizer as well.

#### Apodized Lyot coronagraph (APLC)

For this one, we set up exactly the same Lyot coronagraph as above, but we also need to add the Apodizer, which we can really use as our telescope aperture. To do that,  if you read it in from a fits file, you indeed need to put it on a grid - basically create a grid, and do  
```py
apod = Field(read_fits('apodizer.fits').ravel(), grid)
```

Then turn it into an apodizer (`apodizer = Apodizer(apod)`).

In [ ]:
apod_fname = '/Users/ilaginja/Documents/Git/LUVOIR-modeling/analytical/input_data/apod.fits'
apod = Field(np.rot90(read_fits(apod_fname)).ravel(), pupil_grid)

In [ ]:
imshow_field(apod)

In [ ]:
# Overlap the apodizer iwth the aperture
imshow_field(apod + aper)

They match pretty well, except that the apodizer has supposedly wider segment gaps than the direct aperture. We can now propagate this through the Lyot coronagraph nad get an APLC image.

In [ ]:
lam_over_d = wvln.to(u.m).value / diam  # rad
print('lambda/D = {} rad'.format(lam_over_d))

# Create FPM on a focal grid, with radius in lambda/D, converted to radians
fpm = 1 - circular_aperture(8*lam_over_d)(focal_grid)
print(fpm.shaped.shape)

# Create Lyot coronagraph on pupil grid with FPM
coro_lyot = LyotCoronagraph(pupil_grid, fpm)

In [ ]:
# Create an Apodizer() object for the HiCAT apodizer
apodizer = Apodizer(apod)

# Propagate the wf coming from the SM through the apodizer
aplc_entry = apodizer(wf_post_sm)

In [ ]:
# Propagate from the wavefront at the apodizer to Lyot plane
lyot_plane = coro_lyot(aplc_entry)

# Create Lyot stop
aperture = circular_aperture(0.83 * diam)
obscuration = circular_aperture(0.2 * diam)
res = aperture(pupil_grid) - obscuration(pupil_grid)
ls = Field(res, pupil_grid)

# Propagate to final image
lyot_stop = Apodizer(aperture(pupil_grid) - obscuration(pupil_grid))

# Apply the Lyot stop to the Lyot plane and propagate to the image plane
im_coro_lyot = prop(lyot_stop(lyot_plane))

# Display the coro image
imshow_field(np.log10(im_coro_lyot.intensity / normp))
plt.title("APLC, no aberrations")
plt.colorbar()

### A single aberrated segment

For the calibration of the analytical images, I need to create images that stem for the pupil having one single aberrated segment.

In [ ]:
# Define what segment to aberrate
segnum = 5     # Which segment are we aberrating - I number them starting with 1
segnum -= 1    # Which is why I have to subtract one, because WebbPSF starts numbering them at 0
# Extract the correct segment name from WebbPSF
seg = wss_segs[segnum].split('-')[0]
print('Aberrated segment:', seg)

# Define what Noll Zernike we're using
zern_number = 1
wss_zern_nb = util.noll_to_wss(zern_number)  

# Maybe play around with amount of aberration
#nm_aber = 1000.     # in input units

# Create arrays to hold Zernike aberration coefficients
Aber_WSS = np.zeros([nb_seg, zern_max])           # The Zernikes here will be filled in the WSS order!!!
                                                  # Because it goes into _apply_hexikes_to_seg().

# Feed the aberration nm_aber into the array position
# that corresponds to the correct Zernike, but only on segment i
Aber_WSS[segnum, wss_zern_nb-1] = nm_aber.to(u.m).value     # Aberration on the segment we're currently working on;
                                                            # convert to meters; -1 on the Zernike because Python starts
                                                            # numbering at 0.

#-# Crate OPD with aberrated segment, NO CORONAGRAPH
print('Applying aberration to OTE.')
print('nm_aber: {}'.format(nm_aber))
ote_coro.reset()   # Making sure there are no previous movements on the segments.
ote_coro.zero()    # For now, ignore internal WFE.
ote_coro._apply_hexikes_to_seg(seg, Aber_WSS[segnum,:])

# Display the OTE
ote_coro.display_opd()
plt.show()
# At this point, WebbPSF still numbers the segments wrong in the exit pupil,
# so it's the easiest to orient yourself by the spiders.

In [ ]:
# Calculate the PSF
psf_single_coro = nc_coro.calc_psf(fov_pixels=im_size_e2e, oversample=1, nlambda=1)
plt.figure(figsize=(10, 10))
webbpsf.display_psf(psf_single_coro, vmin=1e-12, vmax=1e-6)
plt.show()

psf_single_coro = psf_single_coro[1].data/normp

In [ ]:
# Display with matplotlib
boxhw = im_size_e2e/2
box2 = boxhw/2
print('nm_aber: {}'.format(nm_aber))

plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(psf_single_coro, norm=LogNorm(), origin='lower')
plt.title('One aberrated segment in coronagraphic setup')
plt.colorbar()
plt.subplot(1, 2, 2)
plt.imshow(util.zoom_cen(psf_single_coro, box2), norm=LogNorm(), origin='lower')
plt.title('Zoomed in')
plt.show()

For piston, an aberration of 10 nm shows no effect, 100 nm already visibly messes the PSF up and 1000 nm make a very distinct change to the PSF, probably too much for PASTIS purposes.

### Pair-wise aberrated segments with coronagraph

In [ ]:
# Decide which two segments you want to aberrate
segnum1 = 8     # Which segments are we aberrating - I number them starting with 1
segnum2 = 16

# Segment aberrations are additive, so if you use a segment number twice, the
# aberration will be applied twice!

segnum_array = np.array([segnum1, segnum2])
segnum_array -= 1    # Which is why I have to subtract one, because WebbPSF starts numbering them at 0

zern_pair = 1  # Which Noll Zernike are we putting on the segments.

# Extract the correct segment names from WebbPSF
seg_array = []
for i, senu in enumerate(segnum_array):
    seg_array.append(wss_segs[senu].split('-')[0])

seg_array = np.array(seg_array)
print('Aberration used: {}'.format(nm_aber))
print('Aberrated segments:', seg_array)
print('Noll Zernike used:', zern_pair)

In [ ]:
aber_wss_loop = np.zeros([nb_seg, 8])
noll_as_wss = np.array([1, 3, 2, 5, 4, 6, 7, 8]) #, 11, 9, 10])    # reordering Noll Zernikes to WSS, for ease of use
print('nm_aber: {}'.format(nm_aber))

# Apply aberration to all sgements
ote_coro.reset()   # Making sure there are no previous movements on the segments.
ote_coro.zero()    # For now, ignore internal WFE.
for i, nseg in enumerate(seg_array):
    aber_wss_loop[segnum_array[i], noll_as_wss[zern_pair-1]-1] = nm_aber.to(u.m).value   # fill only the index for current Zernike, in meters

    # Put Zernike on correct segments on OTE
    ote_coro._apply_hexikes_to_seg(nseg, aber_wss_loop[segnum_array[i],:])

# Display the OTE
ote_coro.display_opd()
plt.show()

In [ ]:
# Calculate the PSF
psf_coro_pair= nc_coro.calc_psf(fov_pixels=im_size_e2e, oversample=1, nlambda=1)
psf_coro_pair = psf_coro_pair[0].data/normp                 # getting the oversampled extension
"""   
# Display the PSF
plt.figure(figsize=(20, 10))
plt.subplot(1, 2, 1)
plt.imshow(psf_coro_pair, norm=LogNorm(), origin='lower')
plt.title('Pair-wise aberrated coronagraphic PSF')
plt.subplot(1, 2, 2)
plt.imshow(util.zoom_cen(psf_coro_pair, box2), norm=LogNorm(), origin='lower')
plt.title('Zoomed')
plt.show()
"""
print('nm_aber: {}'.format(nm_aber))
print('Aberrated segments:', seg_array)
print('Noll Zernike used:', zern_pair)
print(psf_coro_pair.shape)

In [ ]:
# Create DH
dh_area = util.create_dark_hole(psf_coro_pair, inner_wa, outer_wa, sampling)

testim = psf_coro_pair * dh_area

#
contrast = np.mean(testim[np.where(testim != 0)])
print(contrast)

plt.imshow(testim)
plt.show()

In [ ]:
#segs_3_11_noll_1_coro = np.copy(psf_coro_pair)
#segs_11_17_noll_1_coro = np.copy(psf_coro_pair)
#segs_6_11_noll_1_coro = np.copy(psf_coro_pair)
#segs_9_2_noll_1_coro = np.copy(psf_coro_pair)
#segs_9_5_noll_1_coro = np.copy(psf_coro_pair)
#segs_9_15_noll_1_coro = np.copy(psf_coro_pair)
#segs_8_1_noll_1_coro = np.copy(psf_coro_pair)
#segs_8_6_noll_1_coro = np.copy(psf_coro_pair)
#segs_8_16_noll_1_coro = np.copy(psf_coro_pair)

In [ ]:
# Save to central store
save_dir1 = '/astro/opticslab1/PASTIS/jwst_data/E2E_pair_aberrations/2019-1-22-9h-53min'
#util.write_fits(segs_3_11_noll_1_coro, os.path.join(save_dir1, 'segs_3_11_noll_1_coro.fits'))
#util.write_fits(segs_11_17_noll_1_coro, os.path.join(save_dir1, 'segs_11_17_noll_1_coro.fits'))
#util.write_fits(segs_6_11_noll_1_coro, os.path.join(save_dir1, 'segs_6_11_noll_1_coro.fits'))
#util.write_fits(segs_9_2_noll_1_coro, os.path.join(save_dir1, 'segs_9_2_noll_1_coro.fits'))
#util.write_fits(segs_9_5_noll_1_coro, os.path.join(save_dir1, 'segs_9_5_noll_1_coro.fits'))
#util.write_fits(segs_9_15_noll_1_coro, os.path.join(save_dir1, 'segs_9_15_noll_1_coro.fits'))

#util.write_fits(segs_8_1_noll_1_coro, os.path.join(save_dir1, 'segs_8_1_noll_1_coro.fits'))
#util.write_fits(segs_8_6_noll_1_coro, os.path.join(save_dir1, 'segs_8_6_noll_1_coro.fits'))
#util.write_fits(segs_8_16_noll_1_coro, os.path.join(save_dir1, 'segs_8_16_noll_1_coro.fits'))

In [ ]:
# Read from central store

# 1000 nm aberrations:
read_dir1 = '/astro/opticslab1/PASTIS/jwst_data/E2E_pair_aberrations/2019-1-22-9h-53min_coro_piston_1000nm_pairs'

# 100 nm aberrations
#read_dir1 = '/astro/opticslab1/PASTIS/jwst_data/E2E_pair_aberrations/2019-1-25-16h-18min_piston_100nm'

segs_3_11_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_3_11_noll_1.fits'))
segs_11_17_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_11_17_noll_1.fits'))
segs_6_11_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_6_11_noll_1.fits'))
segs_9_2_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_9_2_noll_1.fits'))
segs_9_5_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_9_5_noll_1.fits'))
segs_9_15_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_9_15_noll_1.fits'))

segs_8_1_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_8_1_noll_1.fits'))
segs_8_6_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_8_6_noll_1.fits'))
segs_8_16_noll_1_coro = fits.getdata(os.path.join(read_dir1, 'segs_8_16_noll_1.fits'))

In [ ]:
# Have a look at the images

# Gotta check how big the loaded images are!
print('Loaded images shape:', segs_3_11_noll_1_coro.shape)
print('im_size_e2e:', im_size_e2e)

# If im_size_e2e is bigger than images we loaded, this won't work
# and you have to define a box half-size manually for imwidth.
boxw = int(im_size_e2e/2)
boxw2 = boxw/2
    
if im_size_e2e < segs_3_11_noll_1_dir.shape[0]:
    imwidth = bozw2
else:
    #raise Exception('! You have to set imwidth manually ! And then comment this line out.')
    pass

# Chose what image size (in pixels) we want to display
imwidth = 50

plt.figure(figsize=(18, 12))
plt.suptitle('Pair-wise aberration in coronagraphpic WebbPSF images')
plt.subplot(2, 3, 1)
plt.imshow(util.zoom_cen(segs_3_11_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 3 and 11')

plt.subplot(2, 3, 2)
plt.imshow(util.zoom_cen(segs_6_11_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 6 and 11')

plt.subplot(2, 3, 3)
plt.imshow(util.zoom_cen(segs_11_17_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 11 and 17')

plt.subplot(2, 3, 4)
plt.imshow(util.zoom_cen(segs_9_2_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 9 and 2')

plt.subplot(2, 3, 5)
plt.imshow(util.zoom_cen(segs_9_5_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 5 and 9')

plt.subplot(2, 3, 6)
plt.imshow(util.zoom_cen(segs_9_15_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 9 and 15')

plt.show()

We're missing one intermediate baseline with these combinations though, because we have to skip the center segment. I want to know what that looks like though, so here's some more images.

In [ ]:
# Chose what image size (in pixels) we want to display
imwidth = imwidth

plt.figure(figsize=(18, 6))
plt.suptitle('Pair-wise aberrated coronagraphpic WebbPSF images')
plt.subplot(1, 3, 1)
plt.imshow(util.zoom_cen(segs_8_1_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 8 and 1')

plt.subplot(1, 3, 2)
plt.imshow(util.zoom_cen(segs_8_6_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 8 and 6')

plt.subplot(1, 3, 3)
plt.imshow(util.zoom_cen(segs_8_16_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Piston on segments 8 and 16')

plt.show()

## COMPARING ANALYTICAL, E2E DIRECT AND E2E CORONAGRAPHIC

Display comparison for **piston** with the pairs **9-2**, **9-5** and **9-15**.

In [ ]:
# Chose what image size (in pixels) we want to display
imwidth = imwidth

plt.figure(figsize=(18, 18))
plt.suptitle('Comparison of pair-wise aberration')
plt.subplot(3, 3, 1)
plt.imshow(util.zoom_cen(segs_9_2_noll_1_ana, imwidth), norm=LogNorm(), origin='lower')
plt.title('Analytical - Piston on segments 9 and 2')

plt.subplot(3, 3, 2)
plt.imshow(util.zoom_cen(segs_9_5_noll_1_ana, imwidth), norm=LogNorm(), origin='lower')
plt.title('Analytical - Piston on segments 5 and 9')

plt.subplot(3, 3, 3)
plt.imshow(util.zoom_cen(segs_9_15_noll_1_ana, imwidth), norm=LogNorm(), origin='lower')
plt.title('Analytical - Piston on segments 9 and 15')

plt.subplot(3, 3, 4)
plt.imshow(util.zoom_cen(segs_9_2_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Direct WebbPSF - Piston on segments 9 and 2')

plt.subplot(3, 3, 5)
plt.imshow(util.zoom_cen(segs_9_5_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Direct WebbPSF - Piston on segments 5 and 9')

plt.subplot(3, 3, 6)
plt.imshow(util.zoom_cen(segs_9_15_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Direct WebbPSF - Piston on segments 9 and 15')

plt.subplot(3, 3, 7)
plt.imshow(util.zoom_cen(segs_9_2_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Coro WebbPSF - Piston on segments 9 and 2')

plt.subplot(3, 3, 8)
plt.imshow(util.zoom_cen(segs_9_5_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Coro WebbPSF - Piston on segments 5 and 9')

plt.subplot(3, 3, 9)
plt.imshow(util.zoom_cen(segs_9_15_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Coro WebbPSF - Piston on segments 9 and 15')

plt.show()

Display the comparison for **piston** with the pairs **8-1**, **8-6** and **8-16**.

In [ ]:
# Chose what image size (in pixels) we want to display
imwidth = imwidth

plt.figure(figsize=(18, 18))
plt.suptitle('Comparison of pair-wise aberration')
plt.subplot(3, 3, 1)
plt.imshow(util.zoom_cen(segs_8_1_noll_1_ana, imwidth), norm=LogNorm(), origin='lower')
plt.title('Analytical - Piston on segments 8 and 1')

plt.subplot(3, 3, 2)
plt.imshow(util.zoom_cen(segs_8_6_noll_1_ana, imwidth), norm=LogNorm(), origin='lower')
plt.title('Analytical - Piston on segments 8 and 6')

plt.subplot(3, 3, 3)
plt.imshow(util.zoom_cen(segs_8_16_noll_1_ana, imwidth), norm=LogNorm(), origin='lower')
plt.title('Analytical - Piston on segments 8 and 16')

plt.subplot(3, 3, 4)
plt.imshow(util.zoom_cen(segs_8_1_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Direct WebbPSF - Piston on segments 8 and 1')

plt.subplot(3, 3, 5)
plt.imshow(util.zoom_cen(segs_8_6_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Direct WebbPSF - Piston on segments 8 and 6')

plt.subplot(3, 3, 6)
plt.imshow(util.zoom_cen(segs_8_16_noll_1_dir, imwidth), norm=LogNorm(), origin='lower')
plt.title('Direct WebbPSF - Piston on segments 6 and 16')

plt.subplot(3, 3, 7)
plt.imshow(util.zoom_cen(segs_8_1_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Coro WebbPSF - Piston on segments 8 and 1')

plt.subplot(3, 3, 8)
plt.imshow(util.zoom_cen(segs_8_6_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Coro WebbPSF - Piston on segments 8 and 69')

plt.subplot(3, 3, 9)
plt.imshow(util.zoom_cen(segs_8_16_noll_1_coro, imwidth), norm=LogNorm(), origin='lower')
plt.title('Coro WebbPSF - Piston on segments 8 and 18')

plt.show()